# Exemplo 06: Processamento Paralelo no Spark  
## Cálculo do valor de Pi Serial & Paralelo

Esse exemplo mostra o processamento paralelo no cluster LASID. Para isso, realizamos o cálculo do valor do número Pi usando método Monte Carlo (não é a melhor maneira de calcular, mas exige muito processamento...).

Esse programa define o tamanho do contador (*partition*) e cria vários processos com a função Map que retornam o valor que é totalizado com a função Reduce.

A primeira etapa realiza o cálculo em apenas um host, a segunda etapa realiza o cálculo paralelo com vários hosts usando virtualização da JVM e o terceiro realiza o cálculo paralelo com vários hosts usando virtualização Docker.

Esse exemplo avalia apenas o processamento (operação matemática) e não a leitura de arquivos. 

In [1]:
# Load Python modules
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from random import random
from operator import add
import time

In [2]:
# Pi calculation using Monte Carlo Method
def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

## Spark Pi Serial

In [3]:
# Starting Pi Serial (one processor)
start_time = time.time()

spark = SparkSession.builder\
         .master("local[4]")\
         .appName("JupyterPiSerial")\
         .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/26 18:05:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/26 18:05:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
partitions = 2000
n = 100000 * partitions

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)

print("### Pi is roughly %f ###" % (4.0 * count / n))
print("--- Serial Execution time: %s seconds ---" % (time.time() - start_time))

### Pi is roughly 3.151880 ###
--- Serial Execution time: 54.5758843421936 seconds ---


In [5]:
# Stop Spark session
spark.stop()

## Spark Pi Parallel Standalone Cluster

In [6]:
# Starting Pi Parallel (Spark Cluster with JVM)
start_time = time.time()

spark = SparkSession.builder \
        .master("spark://lasidcluster:7077") \
        .appName("JupyterPiParallel") \
        .config("spark.sql.shuffle.partitions","10000") \
        .config("spark.driver.memory","4g") \
        .config("spark.executor.memory","4g") \
        .getOrCreate()


22/07/26 18:06:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
partitions = 2000
n = 100000 * partitions

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)

print("### Pi is roughly %f ###" % (4.0 * count / n))
print("--- Parallel JVM Execution time: %s seconds ---" % (time.time() - start_time))

### Pi is roughly 3.151880 ###
--- Parallel JVM Execution time: 7.432798147201538 seconds ---


In [8]:
# Stop Spark session
spark.stop()